## PaliGemma 🤝 Direct Preference Optimization

In [2]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!pip install -q git+https://github.com/qgallouedec/trl.git@main

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.4/103.4 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 28.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is t

In [4]:
!pip install -q -U peft accelerate bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 13.1 MB/s eta 0:00:00


Let's load the model and the processor. First we need to login using `notebook_login` since PaliGemma is a gated model. For utmost security, you can create a fine-grained token with access to PaliGemma and pass that.

In [5]:
from huggingface_hub import notebook_login

notebook_login()

In [1]:
from transformers import AutoModelForVision2Seq, AutoProcessor
import torch

model_id = "google/paligemma-3b-pt-448"
model = AutoModelForVision2Seq.from_pretrained(model_id, torch_dtype=torch.bfloat16)
model_ref = AutoModelForVision2Seq.from_pretrained(model_id)
processor = AutoProcessor.from_pretrained(model_id, do_image_splitting=False)
tokenizer = processor.tokenizer

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

If you want to opt for parameter efficient fine-tuning, feel free to run below cell.

In [ ]:
from trl import (
    ModelConfig,
    get_peft_config,
    get_quantization_config
)
from peft import LoraConfig
from transformers import BitsAndBytesConfig
from trl import get_kbit_device_map
from transformers import AutoModelForVision2Seq

model_config = ModelConfig()
# do not run below two lines if you're not doing QLoRA
quantization_config = get_quantization_config(model_config)
device_map=get_kbit_device_map()
peft_config = get_peft_config(model_config)
model = AutoModelForVision2Seq.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map=device_map)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from datasets import load_dataset

ds = load_dataset("HuggingFaceH4/rlaif-v_formatted")

Generating train split:   0%|          | 0/78975 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4157 [00:00<?, ? examples/s]

### Preprocessing

PaliGemma doesn't have a chat template by default, so we need to add it to later preprocess.

In [4]:
import logging
import os
import torch
from datasets import load_dataset

# paligemma has no chat template, we need to add it
processor.chat_template = """{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions. {% for message in messages %}{% if message['role'] == 'user' %}USER: {% else %}ASSISTANT: {% endif %}{% for item in message['content'] %}{% if item['type'] == 'text' %}{{ item['text'] }}{% elif item['type'] == 'image' %}<image>{% endif %}{% endfor %}{% if message['role'] == 'user' %} {% else %}{{eos_token}}{% endif %}{% endfor %}{% if add_generation_prompt %}ASSISTANT: {% endif %}"""


def process(row):
  row["prompt"] = processor.apply_chat_template(row["prompt"], tokenize=False)
  row["chosen"] = processor.apply_chat_template(row["chosen"], tokenize=False)
  row["rejected"] = processor.apply_chat_template(row["rejected"], tokenize=False)
  row["images"][0] = row["images"][0].convert("RGB")
  return row

We'll do a very small run (as this notebook is made for education purposes).

In [5]:
train_dataset = ds["train"].train_test_split(test_size=0.5)["train"]
eval_dataset = ds["test"]

/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=8):   0%|          | 0/39487 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/4157 [00:00<?, ? examples/s]

In [ ]:
from trl import DPOConfig, DPOTrainer, ModelConfig

training_args = DPOConfig(output_dir="/content/outs",
                          per_device_train_batch_size=8,
                          num_train_epochs=1)

trainer = DPOTrainer(
    model,
    model_ref,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor,
    dataset_num_proc=32,
    # uncomment for peft
    # peft_config=get_peft_config(model_config)
)
trainer.train()

trainer.save_model(training_args.output_dir)